In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# --- Global variables for model and vectorizer to avoid retraining on every prediction ---
model = None
vectorize_layer = None
MAX_VOCAB_SIZE = 10000 # Max number of unique words to consider
MAX_SEQUENCE_LENGTH = 128 # Max length of a message after tokenization

def build_and_train_model():
    global model, vectorize_layer

    # Load data
    train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
    test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

    # Convert labels to numerical (ham: 0, spam: 1)
    train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
    test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

    # Prepare data for TensorFlow
    train_messages = train_df['message'].values
    train_labels = train_df['label'].values
    test_messages = test_df['message'].values
    test_labels = test_df['label'].values

    # Create a TextVectorization layer
    vectorize_layer = tf.keras.layers.TextVectorization(
        max_tokens=MAX_VOCAB_SIZE,
        output_mode='int',
        output_sequence_length=MAX_SEQUENCE_LENGTH
    )

    # Adapt the layer to the training data to build the vocabulary
    vectorize_layer.adapt(train_messages)

    # Create the TensorFlow Dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((train_messages, train_labels)).batch(32).prefetch(tf.data.AUTOTUNE)
    test_dataset = tf.data.Dataset.from_tensor_slices((test_messages, test_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

    # Build the model
    model = keras.Sequential([
        vectorize_layer, # Text preprocessing layer
        keras.layers.Embedding(MAX_VOCAB_SIZE, 64, mask_zero=True), # Embedding layer
        # Reverting to Bidirectional LSTM for better sequence understanding
        keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)), # Added return_sequences=True
        keras.layers.Bidirectional(keras.layers.LSTM(32)), # Stacked LSTM for more complexity
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.3), # Adjusted dropout rate
        keras.layers.Dense(1, activation='sigmoid') # Output layer for binary classification
    ])

    # Compile the model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    # Train the model
    print("\nStarting model training...")
    # Increased epochs for better learning
    history = model.fit(train_dataset, epochs=20, validation_data=test_dataset, verbose=1) # verbose=1 to see training progress

    print("Model training complete.")
    loss, accuracy = model.evaluate(test_dataset, verbose=0)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")

# Train the model if it hasn't been built and trained already
if model is None or vectorize_layer is None:
    build_and_train_model()

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  global model, vectorize_layer # Ensure we access global variables

  # This check ensures the model is trained if predict_message is called directly
  # without the main script body running first (e.g., in an interactive environment).
  if model is None or vectorize_layer is None:
      print("Model or vectorizer not loaded. Attempting to build and train the model...")
      build_and_train_model()
      if model is None or vectorize_layer is None: # If training failed
          return [0.5, "Error: Model not available."]


  # The model expects a batch of inputs, even for a single prediction.
  # Use tf.constant to create a tensor from the string.
  prediction_proba = model.predict(tf.constant([pred_text]), verbose=0)[0][0] # verbose=0 to suppress predict output

  # Determine label based on probability
  if prediction_proba > 0.5:
      label = 'spam'
      likeliness = prediction_proba # Likeliness of spam
  else:
      label = 'ham'
      likeliness = prediction_proba # Likeliness of spam (closer to 0 for ham)

  return [likeliness, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(f"\nExample Prediction for '{pred_text}': {prediction}")

pred_text = "sale today! to stop texts call 98912460324"
prediction = predict_message(pred_text)
print(f"\nExample Prediction for '{pred_text}': {prediction}")


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False
      print(f"Failed for message: '{msg}'")
      print(f"Expected: '{ans}', Got: '{prediction[1]}'")

  if passed:
    print("\nYou passed the challenge. Great job!")
  else:
    print("\nYou haven't passed yet. Keep trying.")

test_predictions()